In [ ]:
from matplotlib import pyplot as plt
import os
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook, tnrange
import glob

from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from PIL import Image, ImageDraw, ImageFile

from skimage.external import tifffile


import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam


In [ ]:
data = 'data/xview2/xview2_tier3/'

IMAGE_DIR = os.path.join(data, 'snippet_test_images')
OUTPUT_DIR = os.path.join(data, 'snippet_test_images_dim')

image_glob_paths = os.path.join(IMAGE_DIR, '*_disaster.png')

split_dimension = 128
image_dimension = 1024
image_channels = 3
mask_channels = 1
RGB_bits = 2047  # RGB images
mask_bits = 255  # grayscale

# train_percentage = 1
train_percentage = 0.1
snippet_image_paths = glob.glob(image_glob_paths)

train_index = int((len(snippet_image_paths) * train_percentage) // 1)

if train_percentage != 1:
    snippet_image_paths = snippet_image_paths[:train_index]


print(f'Splitting {len(snippet_image_paths)} images'

In [ ]:
total_ids = image_ids
train_index = int((len(total_ids) * train_percentage) // 1)
ids = image_ids[:train_index]

X = np.zeros((len(ids), image_dimension, image_dimension, image_channels))
y = np.zeros((len(ids), image_dimension, image_dimension, image_channels), dtype=dtype_float)

print('Getting and resizing images ... ')

x_min = 1000
x_max = 0
y_min = 1000
y_max = 0

for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
    # Load images
    image_path = os.path.join(image_dir, f'{image_prefix}{id_}.tif')
    mask_path = os.path.join(output_dir, f'{id_}_mask.tif')
    
    if not (os.path.exists(image_path) and os.path.exists(mask_path)):
        continue
    
    img = tifffile.imread(image_path)
    
    x_img = img
    x_mask = x_img
    x_mask_min = x_mask.min()
    x_mask_max = x_mask.max()
    if x_mask_max > x_max:
        x_max = x_mask_max
    
    if x_mask_min < x_min:
        x_min = x_mask_min
                  
#     x_img = img_to_array(img)
#     x_img = resize(x_img, (image_dimension, image_dimension, image_channels),
#                    mode='constant', preserve_range=True)

    # Load masks
    mask = tifffile.imread(mask_path)
    y_mask = mask
    
    y_mask_min = y_mask.min()
    y_mask_max = y_mask.max()
    if y_mask_max > y_max:
        y_max = y_mask_max
    
    if y_mask_min < y_min:
        y_min = y_mask_min